In [1]:
#!pip install ollama

In [13]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'what country is the krabi province in?',
  },
])
print(response['message']['content'])

The Krabi Province is located in Thailand. It is one of the 77 provinces (changwat) of Thailand and is situated in the southern part of the country, bordering the Andaman Sea to the west and the provinces of Surat Thani and Nakhon Si Thammarat to the north and east respectively.
